In [322]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import datetime

In [323]:
def open_and_split(fileName, starting_index):
    dummy = open(fileName, "r")
    length = len(dummy.readlines()) - 1
    dummy.close()
    
    file = open(fileName, "r")
    data = [next(file) for _ in range(length)]
    file.close()

    #divide linens and split the data
    data = [data[i].replace("\n", "") for i in range(len(data))]
    data = [data[i].split() for i in range(len(data))]
    
    #delete first row
    print(f"deleted rows: {data[0]}")
    data = data[starting_index:]
    print(type(data))
    
    return data

    

In [324]:
data = open_and_split("Composite_Index.txt", 1)

#select only necessary columns: Year (1), Day (2), Hour (3), E(1) (5)
selected_data = [[int(data[i][1]), int(data[i][2]), int(data[i][3]), float(data[i][5])] for i in range(len(data))]

selected_data

deleted rows: ['SORT', 'year', 'day', 'hr', 'S(1)', 'E(1)', 'E(1)-E(1)pred', 'Y(1)=log(1+absAL)', 'Y(2)=log(1+absAU)', 'Y(3)=PCI', 'Y(4)=Kp', 'Y(5)=am', 'Y(6)=abs(Dst)', 'Y(7)=log(0.01+Pemean)', 'Y(8)=log(0.01+Pimean)', 'Y(9)=log(0.01+Pips)', 'Y(10)=log(Fe130)', 'Y(11)=exp(-Ssince)', 'Y(12)=abs(Tilt)', 'Y(13)=log(vsw)', 'Y(14)=log(nsw)', 'Y(15)=sin2(theta/2)3', 'Y(16)=-Bz', 'Y(17)=log(Bmag)', 'Y(18)=thetaBn3', 'Y(19)=Rqint10']
<class 'list'>


[[1991, 313, 7, 3.9124],
 [1991, 313, 9, 3.5547],
 [1991, 313, 10, 3.2938],
 [1991, 313, 11, 3.5561],
 [1991, 313, 12, 3.1413],
 [1991, 313, 13, 3.5273],
 [1991, 313, 14, 3.1636],
 [1991, 313, 15, 3.175],
 [1991, 313, 16, 2.6496],
 [1991, 313, 20, 2.7389],
 [1991, 314, 5, 0.8866],
 [1991, 314, 7, 1.4498],
 [1991, 314, 9, 1.5288],
 [1991, 314, 14, 1.1474],
 [1991, 314, 15, 0.96122],
 [1991, 314, 16, 1.2627],
 [1991, 314, 17, 1.6295],
 [1991, 314, 18, 1.7411],
 [1991, 314, 19, 1.7314],
 [1991, 314, 20, 1.6187],
 [1991, 314, 21, 1.4007],
 [1991, 314, 22, 1.3932],
 [1991, 314, 23, 1.1787],
 [1991, 315, 0, 0.94477],
 [1991, 315, 1, 0.44242],
 [1991, 315, 7, 1.4091],
 [1991, 315, 8, 1.0925],
 [1991, 315, 10, 1.2254],
 [1991, 315, 11, 0.54629],
 [1991, 315, 15, 1.66],
 [1991, 315, 16, 1.6932],
 [1991, 315, 17, 1.8404],
 [1991, 315, 22, 2.2386],
 [1991, 315, 23, 2.4398],
 [1991, 316, 0, 1.4587],
 [1991, 316, 11, 0.36904],
 [1991, 316, 12, -0.05505],
 [1991, 316, 13, 0.20272],
 [1991, 316, 14, 

In [325]:
#make default table
E_1_data = pd.DataFrame(selected_data, columns = ["Year", "Day", "Hour", "E(1)"])

E_1_data

,Year,Day,Hour,E(1)
0,1991,313,7,3.912400
1,1991,313,9,3.554700
2,1991,313,10,3.293800
3,1991,313,11,3.556100
4,1991,313,12,3.141300
...,...,...,...,...
102666,2007,365,16,-0.988270
102667,2007,365,17,-0.730520
102668,2007,365,18,-0.346310
102669,2007,365,19,-0.400580


In [326]:
#function that filter year
def filter_year(DataFrame, year):
    condition = DataFrame["Year"] == year
    return DataFrame.where(condition).dropna().reset_index().drop("index", axis = 1).drop("Year", axis=1)

In [327]:
E1_1995 = filter_year(E_1_data, 1995)
E1_1995

,Day,Hour,E(1)
0,1.0,9.0,-0.729050
1,1.0,10.0,-0.420290
2,1.0,11.0,-0.097706
3,1.0,12.0,-0.482400
4,1.0,14.0,-0.794660
...,...,...,...
7671,364.0,19.0,-0.920410
7672,364.0,20.0,-1.243300
7673,364.0,21.0,-1.245700
7674,364.0,22.0,-1.556000


In [328]:
#convert Day-Hour to composite
def D_to_C(DataFrame):
    composite = [int(DataFrame["Day"][i]) * 24 + int(DataFrame["Hour"][i]) for i in range(DataFrame.shape[0])]
    
    new = DataFrame.copy()
    new.insert(0, "Time", composite)
    new = new.drop(["Day", "Hour"], axis = 1)
    
    return new

In [329]:
#test
E1_1995 = D_to_C(E1_1995).groupby("Time").mean()
E1_1995

,E(1)
Time,
33,-0.729050
34,-0.420290
35,-0.097706
36,-0.482400
38,-0.794660
...,...
8755,-0.920410
8756,-1.243300
8757,-1.245700


In [330]:
#DLL file with specific columns with conversion of hr:mm to hr
def dll_convert(DataFrame):
    converted = [[int(DataFrame[i][1]), int(DataFrame[i][2][:2]), float(DataFrame[i][6])] for i in range(len(DataFrame))]
    return converted

In [331]:
#1995 values for DLL
DLL_1995_row = open_and_split("DLL_zenodo_1995_T1h.txt", 21)

DLL_1995_row_selected = dll_convert(DLL_1995_row)
DLL_1995_row_selected

deleted rows: ['****************************************************************************************']
<class 'list'>


[[1, 4, -9.11],
 [1, 4, -9.12],
 [1, 4, -9.12],
 [1, 4, -9.12],
 [1, 4, -9.13],
 [1, 4, -9.15],
 [1, 4, -9.18],
 [1, 4, -9.15],
 [1, 4, -9.13],
 [1, 4, -9.08],
 [1, 4, -9.08],
 [1, 4, -9.08],
 [1, 4, -9.08],
 [1, 6, -9.53],
 [1, 6, -9.54],
 [1, 6, -9.55],
 [1, 6, -9.56],
 [1, 6, -9.56],
 [1, 6, -9.56],
 [1, 6, -9.56],
 [1, 6, -9.58],
 [1, 6, -9.58],
 [1, 6, -9.59],
 [1, 6, -9.6],
 [1, 6, -9.61],
 [1, 6, -9.62],
 [1, 6, -9.63],
 [1, 7, -9.66],
 [1, 7, -9.64],
 [1, 7, -9.59],
 [1, 7, -9.59],
 [1, 7, -9.56],
 [1, 7, -9.55],
 [1, 7, -9.53],
 [1, 7, -9.41],
 [1, 7, -9.43],
 [1, 7, -9.41],
 [1, 7, -9.38],
 [1, 7, -9.35],
 [1, 7, -9.31],
 [1, 7, -9.24],
 [1, 7, -9.12],
 [1, 7, -9.0],
 [1, 7, -8.92],
 [1, 7, -8.88],
 [1, 7, -8.81],
 [1, 7, -8.8],
 [1, 7, -8.81],
 [1, 7, -8.83],
 [1, 7, -8.89],
 [1, 7, -8.93],
 [1, 7, -8.93],
 [1, 7, -8.89],
 [1, 7, -8.81],
 [1, 7, -8.76],
 [1, 7, -8.73],
 [1, 7, -8.61],
 [1, 7, -8.56],
 [1, 7, -8.54],
 [1, 7, -8.54],
 [1, 7, -8.54],
 [1, 7, -8.55],
 [1, 7, -8.

In [332]:
DLL_1995 = pd.DataFrame(DLL_1995_row_selected, columns=["Day", "Hour", "DLL"])
DLL_1995 = D_to_C(DLL_1995)
DLL_1995

,Time,DLL
0,28,-9.11
1,28,-9.12
2,28,-9.12
3,28,-9.12
4,28,-9.13
...,...,...
408107,8783,-8.55
408108,8783,-8.48
408109,8783,-8.41
408110,8783,-8.38


In [333]:
#group by time
grouped_DLL_1995 = DLL_1995.groupby("Time").mean()
grouped_DLL_1995

,DLL
Time,
28,-9.117692
30,-9.576429
31,-8.836842
32,-8.737458
35,-7.578333
...,...
8779,-7.786724
8780,-8.430000
8781,-8.360167


In [334]:
combined_1995 = pd.merge(left=E1_1995, right=grouped_DLL_1995, left_on="Time", right_on="Time").reset_index()
combined_1995

,Time,E(1),DLL
0,35,-0.097706,-7.578333
1,36,-0.482400,-7.760000
2,38,-0.794660,-8.014667
3,39,-0.944370,-8.521724
4,40,-1.011900,-9.042500
...,...,...,...
7135,8755,-0.920410,-8.743559
7136,8756,-1.243300,-8.994561
7137,8757,-1.245700,-8.497759
7138,8758,-1.556000,-9.122778


In [335]:
def standard_units(DataFrame):
    mean_E1 = DataFrame["E(1)"].mean()
    std_E1 = np.std(DataFrame["E(1)"])
    mean_DLL = DataFrame["DLL"].mean()
    std_DLL = np.std(DataFrame["DLL"])
    
    std1 = [((DataFrame["E(1)"][i] - mean_E1) / std_E1) for i in range(len(DataFrame))]
    std2 = [((DataFrame["DLL"][i] - mean_DLL) / std_DLL) for i in range(len(DataFrame))]
    
    return pd.DataFrame({"E(1) STD": std1, "DLL STD": std2})
    

In [336]:
std_1995 = standard_units(combined_1995)
std_1995

,E(1) STD,DLL STD
0,-0.072689,1.016465
1,-0.453194,0.761330
2,-0.762053,0.403672
3,-0.910133,-0.308448
4,-0.976928,-1.039834
...,...,...
7135,-0.886434,-0.619997
7136,-1.205808,-0.972508
7137,-1.208182,-0.274790
7138,-1.515103,-1.152577


In [337]:
def correlation_coefficient(dataframe):
    return np.mean(dataframe["E(1) STD"] * dataframe["DLL STD"])

In [338]:
r = []
val_1995 = correlation_coefficient(std_1995)

r.append(val_1995)
val_1995

0.5454285728501072

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=28a83fbe-9ed8-4ad0-ab71-9db262ace005' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>